In [1]:
import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme.expressions import Beta, Variable, log, exp
from biogeme import models
from biogeme import results as res

In [2]:
#data_file = "http://transp-or.epfl.ch/data/lpmc.dat"
data_file='lpmc10.dat'
lpmc = pd.read_csv(data_file, sep='\t')
lpmc

,trip_id,household_id,person_n,trip_n,travel_mode,purpose,fueltype,faretype,bus_scale,survey_year,...,dur_pt_access,dur_pt_rail,dur_pt_bus,dur_pt_int,pt_interchanges,dur_driving,cost_transit,cost_driving_fuel,cost_driving_ccharge,driving_traffic_percent
0,20,5,1,0,4,3,1,5,0.0,1,...,0.381667,0.000000,0.062222,0.000000,0,0.117222,0.00,0.41,0.0,0.097156
1,41,9,3,0,4,3,1,5,0.0,1,...,0.146944,0.000000,0.225000,0.000000,0,0.200833,0.00,0.48,0.0,0.378976
2,69,13,2,1,4,3,1,1,1.0,1,...,0.029444,0.083333,0.735833,0.398056,3,0.716944,6.00,2.16,0.0,0.582720
3,102,20,2,0,2,3,1,1,1.0,1,...,0.339722,0.183333,0.116667,0.266667,1,0.250833,3.00,0.89,0.0,0.170543
4,105,21,0,1,4,3,1,1,1.0,1,...,0.126389,0.000000,0.150000,0.000000,0,0.125833,1.50,0.37,0.0,0.154525
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,80998,17591,0,5,4,3,1,5,0.0,3,...,0.189167,0.000000,0.117778,0.000000,0,0.082500,0.00,0.22,10.5,0.239057
4996,81000,17592,0,0,3,3,6,5,0.0,3,...,0.105278,0.000000,0.220278,0.000000,0,0.213611,0.00,0.52,0.0,0.412224
4997,81015,17597,0,3,4,3,1,5,0.0,3,...,0.343056,0.000000,0.177500,0.000000,0,0.189444,0.00,0.76,0.0,0.086510
4998,81041,17604,2,4,3,1,1,2,0.0,3,...,0.344444,0.316667,0.000000,0.083333,1,0.386111,1.05,0.98,0.0,0.340288


In [3]:
database = db.Database('trips', lpmc)

In [4]:
trip_id = Variable('trip_id')
household_id = Variable('household_id')
person_n = Variable('person_n')
travel_mode = Variable('travel_mode')
purpose = Variable('purpose')
fueltype = Variable('fueltype')
faretype = Variable('faretype')
bus_scale = Variable('bus_scale')
survey_year = Variable('survey_year')
travel_year = Variable('travel_year')
travel_month = Variable('travel_month')
travel_day = Variable('travel_day')
day_of_week = Variable('day_of_week')
start_time = Variable('day_of_week')
age = Variable('age')
female = Variable('female')
driving_license = Variable('driving_license')
car_ownership = Variable('car_ownership')
distance = Variable
dur_walking = Variable('dur_walking')
dur_cycling = Variable('dur_cycling')
dur_pt_access = Variable('dur_pt_access')
dur_pt_rail = Variable('dur_pt_rail')
dur_pt_bus = Variable('dur_pt_bus')
dur_pt_int = Variable('dur_pt_int')
pt_interchanges = Variable('pt_interchanges')
dur_driving = Variable('dur_driving')
cost_transit = Variable('cost_transit')
cost_driving_fuel = Variable('cost_driving_fuel')
cost_driving_ccharge = Variable('cost_driving_ccharge')
driving_traffic_percent = Variable('driving_traffic_percent')

# Model 0

In [5]:
asc_walk = Beta('asc_walk', 0, None, None, 0)
asc_cycling = Beta('asc_cycling', 0, None, None, 0)
asc_public = Beta('asc_public', 0, None, None, 0)
asc_driving = Beta('asc_driving', 0, None, None, 0)

In [6]:
cost_driving = cost_driving_fuel + cost_driving_ccharge
dur_public = dur_pt_access + dur_pt_rail + dur_pt_bus + dur_pt_int #total duration of public transportation

In [7]:
beta_cost = Beta('beta_cost', 0, None, None, 0)
beta_time = Beta('beta_time', 0, None, None, 0)

In [8]:
V_walk = asc_walk + beta_time * dur_walking
V_cycling = asc_cycling + beta_time * dur_cycling
V_driving = asc_driving + beta_time * dur_driving + beta_cost * cost_driving
V_public = asc_public + beta_time * dur_public + beta_cost * cost_transit

In [9]:
prob_walk = 1 / (1 + exp(V_cycling - V_walk) + exp(V_driving - V_walk) + exp(V_public - V_walk))
prob_cycling = 1 / (1 + exp(V_walk - V_cycling) + exp(V_driving - V_cycling) + exp(V_public - V_cycling))
prob_driving = 1 / (1 + exp(V_cycling - V_driving) + exp(V_walk - V_driving) + exp(V_public - V_driving))
prob_public = 1-prob_driving + prob_cycling + prob_walk

prob_observation = prob_walk * (travel_mode == 1) + prob_cycling * (travel_mode == 2) + prob_driving * (travel_mode == 3) + prob_public * (travel_mode == 4) 
logprob = log(prob_observation)

In [10]:
biogeme = bio.BIOGEME(database, logprob)
biogeme.modelName = 'model_base'

In [11]:
results0 = biogeme.estimate()

[16:17:38] < Warning >   Cannot read file __model_base.iter. Statement is ignored.


In [12]:
print(results0.printGeneralStatistics())

Number of estimated parameters:	6
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-3511.416
Final log likelihood:	-1907.476
Likelihood ratio test for the init. model:	3207.88
Rho-square for the init. model:	0.457
Rho-square-bar for the init. model:	0.455
Akaike Information Criterion:	3826.952
Bayesian Information Criterion:	3866.056
Final gradient norm:	1.4237E-02
Nbr of threads:	16



In [13]:
results0.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycling,2.169646,0.048636,44.610294,0.000000e+00
asc_driving,2.514422,0.046437,54.147531,0.000000e+00
asc_public,-8.666554,0.023001,-376.792636,0.000000e+00
asc_walk,3.982486,0.041616,95.696009,0.000000e+00
beta_cost,0.125126,0.009580,13.060913,0.000000e+00
beta_time,-1.249388,0.150804,-8.284875,2.220446e-16


In [14]:
results0.data.htmlFileName

'model_base.html'

### Define Model 0

In [15]:
V_walk = beta_time * dur_walking
V_cycling = beta_time * dur_cycling
V_driving = beta_time * dur_driving + beta_cost * cost_driving
V_public = beta_time * dur_public + beta_cost * cost_transit
prob_walk = 1 / (1 + exp(V_cycling - V_walk) + exp(V_driving - V_walk) + exp(V_public - V_walk))
prob_cycling = 1 / (1 + exp(V_walk - V_cycling) + exp(V_driving - V_cycling) + exp(V_public - V_cycling))
prob_driving = 1 / (1 + exp(V_cycling - V_driving) + exp(V_walk - V_driving) + exp(V_public - V_driving))
prob_public = 1-prob_driving + prob_cycling + prob_walk

prob_observation = prob_walk * (travel_mode == 1) + prob_cycling * (travel_mode == 2) + prob_driving * (travel_mode == 3) + prob_public * (travel_mode == 4) 
logprob = log(prob_observation)
biogeme = bio.BIOGEME(database, logprob)
biogeme.modelName = 'model_0'
results0 = biogeme.estimate()
print(results0.printGeneralStatistics())
results0.getEstimatedParameters()

Number of estimated parameters:	2
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-3343.264
Final log likelihood:	-3343.264
Likelihood ratio test for the init. model:	-0
Rho-square for the init. model:	0
Rho-square-bar for the init. model:	-0.000598
Akaike Information Criterion:	6690.528
Bayesian Information Criterion:	6703.562
Final gradient norm:	5.5087E-05
Nbr of threads:	16



,Value,Rob. Std err,Rob. t-test,Rob. p-value
beta_cost,0.112753,0.007906,14.262435,0.000000e+00
beta_time,-0.400235,0.063858,-6.267545,3.667857e-10


# Model 1


In [16]:
asc_walk = Beta('asc_walk', 0, None, None, 0)
asc_cycling = Beta('asc_cycling', 0, None, None, 0)
asc_public = Beta('asc_public', 0, None, None, 0)
asc_driving = Beta('asc_driving', 0, None, None, 0)

In [17]:
cost_driving = cost_driving_fuel + cost_driving_ccharge
dur_public = dur_pt_access + dur_pt_rail + dur_pt_bus + dur_pt_int #total duration of public transportation

In [18]:
beta_time_walk = Beta('beta_time_walk', 0, None, None, 0)
beta_time_cycling = Beta('beta_time_cycling', 0, None, None, 0)
beta_time_driving = Beta('beta_time_driving', 0, None, None, 0)
beta_time_public = Beta('beta_time_public', 0, None, None, 0)

In [19]:
'''V_walk = asc_walk + beta_time_walk * dur_walking
V_cycling = asc_cycling + beta_time_cycling * dur_cycling
V_driving = asc_driving + beta_time_driving * dur_driving + beta_cost * cost_driving
V_public = asc_public + beta_time_public * dur_public + beta_cost * cost_transit'''
V_walk = beta_time_walk * dur_walking
V_cycling =  beta_time_cycling * dur_cycling
V_driving = beta_time_driving * dur_driving + beta_cost * cost_driving
V_public = beta_time_public * dur_public + beta_cost * cost_transit

In [20]:
prob_walk = 1 / (1 + exp(V_cycling - V_walk) + exp(V_driving - V_walk) + exp(V_public - V_walk))
prob_cycling = 1 / (1 + exp(V_walk - V_cycling) + exp(V_driving - V_cycling) + exp(V_public - V_cycling))
prob_driving = 1 / (1 + exp(V_cycling - V_driving) + exp(V_walk - V_driving) + exp(V_public - V_driving))
prob_public = 1-prob_driving + prob_cycling + prob_walk

prob_observation = prob_walk * (travel_mode == 1) + prob_cycling * (travel_mode == 2) + prob_driving * (travel_mode == 3) + prob_public * (travel_mode == 4) 
logprob = log(prob_observation)

In [21]:
biogeme = bio.BIOGEME(database, logprob)
biogeme.modelName = 'model_1'

In [22]:
results1 = biogeme.estimate()

In [23]:
print(results1.printGeneralStatistics())

Number of estimated parameters:	5
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-2259.084
Final log likelihood:	-2259.084
Likelihood ratio test for the init. model:	3.711857e-07
Rho-square for the init. model:	8.22e-11
Rho-square-bar for the init. model:	-0.00221
Akaike Information Criterion:	4528.169
Bayesian Information Criterion:	4560.755
Final gradient norm:	1.0925E-02
Nbr of threads:	16



In [24]:
results1.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
beta_cost,0.062682,0.010595,5.916288,3.292893e-09
beta_time_cycling,1.213228,0.298762,4.060847,4.889494e-05
beta_time_driving,3.520233,0.365177,9.639787,0.000000e+00
beta_time_public,-105.310229,4.531871,-23.237694,0.000000e+00
beta_time_walk,0.688292,0.083943,8.199481,2.220446e-16


In [25]:
results1.data.htmlFileName

'model_1.html'

## Comparing Models 0 and 1

In [26]:
general_statistics_model_0 = results0.getGeneralStatistics()
general_statistics_model_0

{'Number of estimated parameters': GeneralStatistic(value=2, format=''),
 'Sample size': GeneralStatistic(value=5000, format=''),
 'Excluded observations': GeneralStatistic(value=0, format=''),
 'Init log likelihood': GeneralStatistic(value=-3343.263889666974, format='.7g'),
 'Final log likelihood': GeneralStatistic(value=-3343.263889666974, format='.7g'),
 'Likelihood ratio test for the init. model': GeneralStatistic(value=-0.0, format='.7g'),
 'Rho-square for the init. model': GeneralStatistic(value=0.0, format='.3g'),
 'Rho-square-bar for the init. model': GeneralStatistic(value=-0.0005982178093035806, format='.3g'),
 'Akaike Information Criterion': GeneralStatistic(value=6690.527779333948, format='.7g'),
 'Bayesian Information Criterion': GeneralStatistic(value=6703.56216571678, format='.7g'),
 'Final gradient norm': GeneralStatistic(value=5.508659263845065e-05, format='.4E'),
 'Nbr of threads': GeneralStatistic(value=16, format='')}

In [27]:
general_statistics_model_1 = results1.getGeneralStatistics()
general_statistics_model_1

{'Number of estimated parameters': GeneralStatistic(value=5, format=''),
 'Sample size': GeneralStatistic(value=5000, format=''),
 'Excluded observations': GeneralStatistic(value=0, format=''),
 'Init log likelihood': GeneralStatistic(value=-2259.084337991777, format='.7g'),
 'Final log likelihood': GeneralStatistic(value=-2259.084337806184, format='.7g'),
 'Likelihood ratio test for the init. model': GeneralStatistic(value=3.711857061716728e-07, format='.7g'),
 'Rho-square for the init. model': GeneralStatistic(value=8.215406133160741e-11, format='.3g'),
 'Rho-square-bar for the init. model': GeneralStatistic(value=-0.0022132860337795712, format='.3g'),
 'Akaike Information Criterion': GeneralStatistic(value=4528.168675612368, format='.7g'),
 'Bayesian Information Criterion': GeneralStatistic(value=4560.75464156945, format='.7g'),
 'Final gradient norm': GeneralStatistic(value=0.010924681985284042, format='.4E'),
 'Nbr of threads': GeneralStatistic(value=16, format='')}

### By hand

In [28]:
alpha = 0.05 #significance level

In [29]:
#compute the log-likelihood ratio
L0=general_statistics_model_0['Final log likelihood'].value
L1=general_statistics_model_1['Final log likelihood'].value
test = -2 * (L0 - L1)
test

2168.35910372158

In [30]:
#compute the degrees of freedom for the chi2 distribution
K0 = general_statistics_model_0['Number of estimated parameters'].value
K1 = general_statistics_model_1['Number of estimated parameters'].value
degrees_of_freedom = K1 - K0
degrees_of_freedom

3

In [31]:
from scipy.stats import chi2
threshold = chi2.ppf(1-alpha, degrees_of_freedom)
threshold

7.814727903251179

2168.3591033503944 > 7.814727903251179 so we reject the null hypothesis at level 0.05%.

### Automatically

In [32]:
results1.likelihood_ratio_test(results0, alpha)

LRTuple(message='H0 can be rejected at level 5.0%', statistic=2168.35910372158, threshold=7.814727903251179)

The null hypothesis is rejected at the 5% level.